In [42]:
import pandas as pd
import re

In [8]:
import pandas as pd 
from Functions.NLP.alertp1_nlp import *
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *

data = pd.read_csv("downloads/modified_dataset_vanilla.csv")

#changing columns with date to a datetime
date_format_alertP1(data)
#change the blank values in proveniencia and priority to unknown
replace_blank(data)
#create a column with the output of a referral and select the refused and accepted ones
data=result(data)
data=data.sort_values('DATA_RETORNO')
data = data.dropna(subset=['Texto'])
data = data.drop('clean_text', axis=1)

# compute the number of rows to select
n = int(len(data) * 0.8)

# select the first 80% of rows using slicing
train = data.iloc[:n]
#select the last 20%
test = data.iloc[n:]
#n+1 so I don't have an overlapp in the distribution

lower_text(train,"Texto","Texto")

random_rows = train.sample(20, random_state= 20)

/Users/miguelcosta/Desktop/PBL/PBL-HGO-1/Functions/AlertP1/data_cleaning.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
/Users/miguelcosta/Desktop/PBL/PBL-HGO-1/Functions/AlertP1/data_cleaning.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['CTH_PRIOR'][alertP1['CTH_PRIOR']=='']='unknown'
/Users/miguelcosta/Desktop/PBL/PBL-HGO-1/Functions/NLP/alertp1_nlp.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [45]:
def check_synonyms(excel_file, df, column, threshold, process_all=False, word=None):

    from fuzzywuzzy import fuzz
    import re

    # Load the list of synonyms from the Excel file
    synonyms_df = pd.read_excel(excel_file)

    # Define a function to check if any of the synonyms are present in the text
    def check_text(text, synonyms):
        count = 0
        words = re.findall(r'\b\w+\b', text.lower())
        for word in words:
            # Use fuzzywuzzy to allow for some mistakes in the way words are written
            scores = [fuzz.ratio(word, synonym) for synonym in synonyms]
            if max(scores) >= threshold:
                count += 1
        return count

    if process_all:
        # Process all column names from the Excel file
        for word in synonyms_df.columns:
            synonyms = synonyms_df[word].dropna().str.lower().tolist()
            df[f"count_{word}"] = df[column].apply(lambda x: check_text(x, synonyms))
    else:
        # Process only the given word
        if word is None:
            raise ValueError("Please provide a word to process.")
        if word not in synonyms_df.columns:
            raise ValueError(f"Invalid word: {word}. Available words: {list(synonyms_df.columns)}")
        synonyms = synonyms_df[word].dropna().str.lower().tolist()
        df[f"count_{word}"] = df[column].apply(lambda x: check_text(x, synonyms))

    return df

In [52]:
check_synonyms("Data/synonyms_dict.xlsx", random_rows, "Texto", 80, process_all=True)

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,trata data recusa,resume saída,mês_saida,ano_saida,Texto,result,count_Alteração,count_Agravamento,count_HGO,count_Estável
1175,FAZDZA,ILBIBJD,LSHLLTVO,ZZBHEAFH,3150502,CHARNECA DA CAPARICA,CS/USF,CTH,3.0,Normal,...,2016-02-24,31/03/2016,3,2016,trémulo mais dos membros superiores - parkinso...,1,0,0,0,0
26,DZHBDC,LVEDIM,LRHRLLSR,ZHFBEFHS,3150305,UCSP RAINHA D. LEONOR,CS/USF,CTH,NaN,unknown,...,2015-08-18,18/08/2015,8,2015,doente com 75 anos que sofreu avc isquemico em...,0,0,0,0,0
1441,FAEEHZ,IIVJMJJJJ,LQHLHOOO,BHADAGH,3151575,USF TORRE DA MARINHA,CS/USF,CTH,NaN,unknown,...,2014-04-04,15/05/2014,5,2014,justificação parquinsonismo sindrome demencial...,1,2,0,0,0
528,DGDGEZ,VLMDJEI,LPHRTVVQ,CCEFBEF,3150371,USF S JOAO DO PRAGAL-CS ALMADA,CS/USF,CTH,NaN,unknown,...,2013-12-13,10/01/2014,1,2014,homem de 51 anos com alterações neurológicas q...,1,3,1,2,0
910,EBAGCC,LLCICNB,LQHSRQQP,GEESEZA,3151574,USF CSI SEIXAL,CS/USF,CTH,3.0,Normal,...,2015-05-28,28/05/2015,5,2015,"utente de 32 anos, sem antec. clinicos relevan...",1,2,0,0,0
344,SFZGAE,LINIMN,LOHPTQHO,NaN,0,NaN,OUTRA,unknown,NaN,unknown,...,2012-08-29,29/08/2012,8,2012,parkinsonismo,0,0,0,0,0
372,DSFFGE,LICVID,LQHSVOSU,GSHSGAB,3150506,UCSP COSTA E TRAFARIA,CS/USF,CTH,NaN,unknown,...,2015-06-11,11/06/2015,6,2015,"doente do sexo masculino com 82 anos, analfabe...",0,2,0,0,1
954,FEEZGB,LLCCVIM,LQHHROVH,NaN,0,NaN,OUTRA,SAM,NaN,unknown,...,2014-03-31,22/05/2014,5,2014,homem de 76 anos avaliado em consulta por coli...,1,1,0,0,0
1605,ZDEGDC,MILLVCE,LQHSVLUR,NaN,0,NaN,OUTRA,SAM,NaN,unknown,...,2015-03-30,13/04/2015,4,2015,"doente de 79 anos, com quadro com pelo menos 1...",1,1,1,0,0
1137,FFEFSH,ILBJDLB,LPHQVLLV,NaN,0,NaN,OUTRA,unknown,NaN,unknown,...,2013-10-17,27/06/2014,6,2014,sem informação,1,0,0,0,0


In [46]:
check_synonyms("Data/synonyms_dict.xlsx", train, "Texto", 80, process_all=True)

/var/folders/g5/fkg03q9j52z1zkf7tm78r7pr0000gn/T/ipykernel_16983/2203895755.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"count_{word}"] = df[column].apply(lambda x: check_text(x, synonyms))


,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,trata data recusa,resume saída,mês_saida,ano_saida,Texto,result,count_Alteração,count_Agravamento,count_HGO,count_Estável
630,EGBZZB,VLNMEEM,LOHHHSHT,SSDBHEA,3150502,CHARNECA DA CAPARICA,CS/USF,CTH,NaN,unknown,...,2012-01-10,23/04/2012,4,2012,"doente de 71 anos de idade, com tremor acentua...",1,0,1,0,0
1537,FGSEDD,MIVCNDB,LOHHLSTU,NaN,0,NaN,OUTRA,unknown,NaN,unknown,...,2012-01-11,11/01/2012,1,2012,sem relatorio clinico,0,0,0,0,0
1700,AZHBEB,MILDBCV,LOHHLUQL,NaN,0,NaN,OUTRA,unknown,NaN,unknown,...,2012-01-12,12/01/2012,1,2012,hemiplegia e hemiparesia,1,0,0,0,0
1365,SBBEBZ,IIJCVNC,LOHHLURO,NaN,0,NaN,OUTRA,unknown,NaN,unknown,...,2012-01-13,13/01/2012,1,2012,sem relatorio clinico,0,0,0,0,0
243,SADASC,LLBIJL,LOHHOSQU,NaN,0,NaN,OUTRA,unknown,NaN,unknown,...,2012-01-17,17/01/2012,1,2012,sem relatorio clinico,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680,AHBECF,MILCMIJ,LSHPHLPO,ZDZBAABF,3152401,UCSP CORROIOS,CS/USF,CTH,NaN,unknown,...,2016-05-20,20/05/2016,5,2016,doente de 83 anos hipertenso e com dislipidemi...,0,5,0,0,0
565,DFSFHD,VLNJEDI,LSHOTSQO,ZDZEZBZC,3151671,USF QUINTA DA ROSINHA,CS/USF,CTH,3.0,Normal,...,2016-05-20,20/07/2016,7,2016,"67 anos sem antecedentes relevantes, refere tr...",1,0,0,0,0
1347,FCGCAH,IIJEVBE,LSHPHSPT,ZDZGADBA,3151671,USF QUINTA DA ROSINHA,CS/USF,CTH,NaN,unknown,...,2016-05-20,20/05/2016,5,2016,"doente com ap de hta, dmtipo2, dislipidemia, o...",0,0,0,0,1
1348,FEGBEH,"I,IE+LII",LSHOUULQ,ZDZFGFZG,3150305,UCSP RAINHA D. LEONOR,CS/USF,CTH,3.0,Normal,...,2016-05-20,11/10/2016,10,2016,78anos. sindrome demencial + parkinsonismo ?,1,0,0,0,0


In [26]:
train[["count_Alteração", "count_Agravamento", "count_HGO", "count_Estável"]].sum()

count_Alteração      897
count_Agravamento    258
count_HGO              0
count_Estável        182
dtype: int64

In [47]:
train[["count_Alteração", "count_Agravamento", "count_HGO", "count_Estável"]].sum()

count_Alteração      1040
count_Agravamento     265
count_HGO             149
count_Estável         234
dtype: int64

In [59]:
train.groupby(["result"])["count_Estável"].value_counts(normalize=True)

result  count_Estável
0       0                0.920455
        1                0.073864
        2                0.005682
1       0                0.849198
        1                0.116578
        2                0.023529
        3                0.005348
        5                0.002139
        4                0.001070
        10               0.001070
        12               0.001070
Name: count_Estável, dtype: float64

In [60]:
train.groupby(["result"])["count_HGO"].value_counts(normalize=True)

result  count_HGO
0       0            0.934659
        1            0.059659
        2            0.005682
1       0            0.882353
        1            0.105882
        2            0.008556
        3            0.003209
Name: count_HGO, dtype: float64

In [57]:
train.groupby(["result"])["count_Alteração"].value_counts(normalize=True)

result  count_Alteração
0       0                  0.750000
        1                  0.161932
        2                  0.048295
        3                  0.019886
        4                  0.008523
        5                  0.005682
        7                  0.002841
        9                  0.002841
1       0                  0.541176
        1                  0.231016
        2                  0.103743
        3                  0.054545
        4                  0.036364
        5                  0.014973
        6                  0.006417
        7                  0.006417
        9                  0.003209
        8                  0.002139
Name: count_Alteração, dtype: float64

In [58]:
train.groupby(["result"])["count_Agravamento"].value_counts(normalize=True)

result  count_Agravamento
0       0                    0.889205
        1                    0.096591
        2                    0.014205
1       0                    0.817112
        1                    0.144385
        2                    0.026738
        3                    0.010695
        6                    0.001070
Name: count_Agravamento, dtype: float64

In [44]:
text = "mizaçao tx Antecedentes pessoais: Síndrome do canal cárpico; Início: 04-Abr-2014 dto Hipertiroidismo/tireotoxicose; Início: 01-Jan-2013 Doença de Graves, iodoterapia Enxaqueca; Início: 01-Jan-2013 Outros problemas respiratórios; Início: 01-Jan-2013 Sindrome de Apneia do Sono Excesso de peso (IMC <30); Início: 01-Jan-2013 Hipotiroidismo/mixedema; Início: 01-Jan-2013 Consequente de Iodoterapia, seguido dra isabel Costa HGO"
words = re.findall(r'\b\w+\b', text.lower())

words

['mizaçao',
 'tx',
 'antecedentes',
 'pessoais',
 'síndrome',
 'do',
 'canal',
 'cárpico',
 'início',
 '04',
 'abr',
 '2014',
 'dto',
 'hipertiroidismo',
 'tireotoxicose',
 'início',
 '01',
 'jan',
 '2013',
 'doença',
 'de',
 'graves',
 'iodoterapia',
 'enxaqueca',
 'início',
 '01',
 'jan',
 '2013',
 'outros',
 'problemas',
 'respiratórios',
 'início',
 '01',
 'jan',
 '2013',
 'sindrome',
 'de',
 'apneia',
 'do',
 'sono',
 'excesso',
 'de',
 'peso',
 'imc',
 '30',
 'início',
 '01',
 'jan',
 '2013',
 'hipotiroidismo',
 'mixedema',
 'início',
 '01',
 'jan',
 '2013',
 'consequente',
 'de',
 'iodoterapia',
 'seguido',
 'dra',
 'isabel',
 'costa',
 'hgo']